In [1]:
import pandas as pd
import scanpy as sc
sc.set_figure_params(dpi=100, frameon=False)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


scanpy==1.6.0 anndata==0.7.4 umap==0.4.6 numpy==1.19.2 scipy==1.6.1 pandas==1.2.3 scikit-learn==0.23.2 statsmodels==0.11.1 python-igraph==0.8.3 leidenalg==0.8.3


In [2]:
import os
os.chdir('./../')
from compert.helper import rank_genes_groups_by_cov

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
adata = sc.read('datasets/lincs.h5ad')

In [5]:
adata.obs['condition'] = adata.obs['pert_iname']
adata.obs['cell_type'] = adata.obs['cell_id']
adata.obs['dose_val'] = adata.obs['pert_dose']
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['control'] = (adata.obs['condition'] == 'DMSO').astype(int)

In [6]:
pd.crosstab(adata.obs.condition, adata.obs.cell_type)

cell_type,A375,A549,A673,AGS,ASC,ASC.C,BT20,CD34,CL34,CORL23,...,SW620,SW948,T3M10,THP1,TYKNU,U266,U937,VCAP,WSUDLCL2,YAPC
condition,,,,,,,,,,,,,,,,,,,,,
1B Parent,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2-methoxyestradiol,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
"3,6-dimethoxyflavone",5,3,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3-amino-benzamide,30,61,2,2,0,0,0,0,2,2,...,4,4,2,2,2,0,2,9,2,18
5-methoxy-alpha-methyltryptamine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xanthoxyline,3,5,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,7,0,0
yohimbine,35,16,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,26,0,18
zacopride,9,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,0,0


Calculate differential genes manually, such that the genes are the same per condition.

In [7]:
%%time
sc.tl.rank_genes_groups(
    adata,
    groupby='condition', 
    reference='DMSO',
    rankby_abs=True,
    n_genes=50
)

de_genes = {}
for cond in adata.obs['condition']:
    if cond != 'DMSO':
        df = sc.get.rank_genes_groups_df(adata, group=cond)  # this takes a while
        de_genes[cond] = df['names'][:50].values

... storing 'cov_drug_dose_name' as categorical


CPU times: user 4min 4s, sys: 412 ms, total: 4min 4s
Wall time: 4min 5s


In [8]:
adata.uns['rank_genes_groups_cov'] = {cond: de_genes[cond.split('_')[1]] for cond in adata.obs['cov_drug_dose_name'].unique() if cond.split('_')[1] != 'DMSO'}

In [9]:
adata.obs['split'] = 'train'

# take ood from top occurring perturbations to avoid losing data on low occ ones
ood_idx = sc.pp.subsample(
    adata[adata.obs.condition.isin(list(adata.obs.condition.value_counts().index[1:50]))],
    .1,
    copy=True
).obs.index
adata.obs['split'].loc[ood_idx] = 'ood'

# take test from a random subsampling of the rest
test_idx = sc.pp.subsample(
    adata[adata.obs.split != 'ood'],
    .16,
    copy=True
).obs.index
adata.obs['split'].loc[test_idx] = 'test'

In [10]:
pd.crosstab(adata.obs['split'], adata.obs['condition'])

condition,1B Parent,2-methoxyestradiol,"3,6-dimethoxyflavone",3-amino-benzamide,5-methoxy-alpha-methyltryptamine,ABT-737,AG-490,AG-14361,AICA-ribonucleotide,ALW-II-38-3,...,veliparib,vinburnine,voglibose,wiskostatin,xanthohumol,xanthoxyline,yohimbine,zacopride,zaprinast,zileuton
split,,,,,,,,,,,,,,,,,,,,,
ood,0,0,0,0,0,64,0,0,0,0,...,135,0,0,0,0,0,0,0,0,0
test,2,17,4,59,4,125,23,66,50,89,...,188,24,20,7,9,9,60,26,33,51
train,20,100,30,343,14,583,130,272,249,509,...,936,150,105,58,58,53,266,81,175,279


In [11]:
del(adata.uns['rank_genes_groups'])  # too large

In [12]:
# code compatibility
from scipy import sparse
adata.X = sparse.csr_matrix(adata.X)

In [13]:
sc.write('datasets/lincs.h5ad', adata)

... storing 'split' as categorical
